In [ ]:
#show table in database
postgres=# \dt

#describe table
postgres=# \d tablename

#show top rows in a table
postgres=# TABLE tablename



# 8 Solution: Indexing Exercise

```sql


-- Constraints
ALTER TABLE "authors"
  ADD PRIMARY KEY ("id");

ALTER TABLE "topics"
  ADD PRIMARY KEY("id"),
  ADD UNIQUE ("name"),
  ALTER COLUMN "name" SET NOT NULL;

ALTER TABLE "books"
  ADD PRIMARY KEY ("id"),
  ADD UNIQUE ("isbn"),
  ADD FOREIGN KEY ("author_id") REFERENCES "authors" ("id");

ALTER TABLE "book_topics"
  ADD PRIMARY KEY ("book_id", "topic_id");
-- or ("topic_id", "book_id") instead...?

-- We need to be able to quickly find books and authors by their IDs.
-- Already taken care of by primary keys

-- We need to be able to quickly tell which books an author has written.
CREATE INDEX "find_books_by_author" ON "books" ("author_id");

-- We need to be able to quickly find a book by its ISBN #.
-- The unique constraint on ISBN already takes care of that 
--   by adding a unique index

-- We need to be able to quickly search for books by their titles
--   in a case-insensitive way, even if the title is partial. For example, 
--   searching for "the" should return "The Lord of the rings".
CREATE INDEX "find_books_by_partial_title" ON "books" (
  LOWER("title") VARCHAR_PATTERN_OPS
);

-- For a given book, we need to be able to quickly find all the topics 
--   associated with it.
-- The primary key on the book_topics table already takes care of that 
--   since there's an underlying unique index

-- For a given topic, we need to be able to quickly find all the books 
--   tagged with it.
CREATE INDEX "find_books_by_topic" ON "book_topics" ("topic_id");

```

# 17 Solution: Creating a Complete Schema Exercise


```sql
CREATE TABLE "movies" (
  "id" SERIAL PRIMARY KEY,
  "title" VARCHAR(500), --  Night of the Day of the Dawn of the Son of the Bride of the Return of the Revenge of the Terror of the Attack of the Evil, Mutant, Hellbound, Flesh-Eating Subhumanoid Zombified Living Dead, Part 3
  "description" TEXT
);


CREATE TABLE "categories" (
  "id" SERIAL PRIMARY KEY,
  "name" VARCHAR(50) UNIQUE
);

CREATE TABLE "movie_categories" (
  "movie_id" INTEGER REFERENCES "movies",
  "category_id" INTEGER REFERENCES "categories",
  PRIMARY KEY ("movie_id", "category_id")
);

CREATE TABLE "users" (
  "id" SERIAL PRIMARY KEY,
  "username" VARCHAR(100),
);
CREATE UNIQUE INDEX ON "users" (LOWER("username"));

CREATE TABLE "user_movie_ratings" (
  "user_id" INTEGER REFERENCES "users",
  "movie_id" INTEGER REFERENCES "movies",
  "rating" SMALLINT CHECK ("rating" BETWEEN 0 AND 100),
  PRIMARY KEY ("user_id", "movie_id")
);
CREATE INDEX ON "user_movie_ratings" ("movie_id");

CREATE TABLE "user_category_likes" (
  "user_id" INTEGER REFERENCES "users",
  "category_id" INTEGER REFERENCES "categories",
  PRIMARY KEY ("user_id", "category_id")
);
CREATE INDEX ON "user_category_likes" ("category_id");```

```sql
TRUNCATE TABLE comments;

ALTER TABLE "comments"
    ADD FOREIGN KEY ("user_id") REFERENCES "users" ("id");

INSERT INTO "comments" ("user_id", "content") VALUES (100, 'comment text'), (-5, 'other comment text...');

ALTER TABLE "comment_likes"
    ADD FOREIGN KEY ("user_id") REFERENCES "users";

ALTER TABLE "comment_likes"
    ADD FOREIGN KEY ("comment_id") REFERENCES "comments" ("id");



DROP TABLE "comment_likes";
CREATE TABLE "comment_likes" (
    "user_id" INTEGER REFERENCES "users",
    "comment_id" INTEGER,
    FOREIGN KEY ("comment_id") REFERENCES "comments" ("id")
);

```

# 17 Glossary


| Key Term            | Definition                                                                                                                                                                      |
|---------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Index               | A data structure stored on disk and targeting one or more columns of a table. Due to the way it's sorted, an index can make it fast to find rows in a table when filtering on the target column(s). |
| Composite Index     | An index that targets multiple columns. The order in which the columns are specified matters.                                                                                   |
| Sequential Scan     | A database operation where every row of a table is visited in order to fulfill a particular query.                                                                              |
| Query Planner       | The part of Postgres that looks at a query to be executed, table statistics, and available resources to decide on a Query Plan.                                                  |
| Query Plan          | The ordered series of steps that Postgres will follow in order to fulfill a query.                                                                                              |
| Query Plan Cost     | A unitless measure of how much effort and resources will be required in order to fulfill a query. Only comparable to itself for a given query after making some changes to data and/or indexes. |
| Query Plan Node     | A unit of work in a query plan, i.e. one of the steps required to fulfill a query.                                                                                               |
